In [1]:
import os
import cv2
from flask import Flask, request, jsonify, Blueprint
from werkzeug.utils import secure_filename
from flask import Flask, make_response
from turbojpeg import TurboJPEG
import json

In [2]:
#DEV ONLY
__file__ = 'G:\\python_projects\\guiserver\\'

In [3]:
#CONFIG
_PORT_ = 7001

UPLOAD_PATH = os.path.join(os.path.dirname(__file__), 'images')
_THIS_URL_ = f'http://localhost:{_PORT_}/'

ModelList = {
}

api = Blueprint('api', __name__,template_folder='templates')


In [4]:
print('images path: ', UPLOAD_PATH)

images path:  G:\python_projects\guiserver\images


In [5]:
def init():
    ''' init the progam
        create the dirs
    '''
    # 上传文件夹如果不存在则创建
    if not os.path.exists(UPLOAD_PATH):
        os.mkdir(UPLOAD_PATH)

init()

In [6]:
@api.route('/', methods=['GET', 'POST'])
# @api.route('/api', methods=['GET', 'POST'])
def hello():
   return 'Hello'

In [7]:
@api.route('/hello/<name>/', methods=['GET', 'POST'])
def hello_name(name):
   return 'Hello %s!' % name

In [8]:
@api.route('/images/<image_name>', methods=['POST'])
def get_image(image_name):
    ''' get the image
    '''
    if os.path.exists(os.path.join(UPLOAD_PATH, image_name)):
        print(f'{UPLOAD_PATH}\\{image_name}')
        with open(os.path.join(UPLOAD_PATH, image_name), 'rb') as f:
            image = f.read()
            #创建response对象
            resp = make_response(image)
            #设置response的headers对象
            resp.headers['Content-Type'] = 'image/*'
            return resp
    else:
        return jsonify({
            'code': 404,
            'message': 'image not found'
        })


In [9]:
@api.route('/upload', methods=['POST','OPTIONS'])
def upload_images_from_form_data():
    if request.method == 'OPTIONS':
        req = request
        data = request.get_data().decode('utf-8')
        print(data)
        return jsonify({
            'code': 200,
            'message': 'ok'
        })
    elif request.method == 'POST':
        # print(request.files)
        imgs = request.files.getlist("image")
        # print(imgs)
        urls = []
        # 循环读取上传的文件并保存
        for img in imgs:
            filename = secure_filename(img.filename)
            # print(filename)
            img.save(os.path.join(UPLOAD_PATH, filename))
            msg = f"{_THIS_URL_}images/{filename}"
            urls.append(msg)
        respose = {
            "code": 200,
            "urls": urls
        }
        return jsonify(respose)

In [10]:
# load the ModelHub
import sys
sys.path.append(os.path.join(os.path.dirname(__file__), 'ModelHub'))
from ModelHub import ModelHub, get_image_by_name
modelHub = ModelHub()

Loaded model: ConvNeXt_tiny


In [11]:
def resolve_data_json(data):
    ''' resolve the data json
    '''
    if 'local_image' in data:
        data['image'] = get_image_by_name(data['local_image'])
    if 'url_image' in data:
        raise NotImplementedError('url_image is not implemented')
    return data

In [12]:
def dispatch(model_name, data):
    global modelHub
    ''' dispatch the model
    '''
    return modelHub.process(model_name, data)

@api.route('/ml/dispatch/<model>/<data>', methods=['POST'])
def ondispatch(model, data):
    model = model.strip()
    if model == 'TESTMODEL':
        return jsonify({
            'code': 200,
            'message': 'Test success!'
        })

    if not os.path.exists(os.path.join(UPLOAD_PATH, data)):
        return jsonify({
            'code': 404,
            'message': 'image not found'
        })
    
    return dispatch(model, {
        'image': get_image_by_name(data)
    })

@api.route('/ml/dispatch', methods=['POST'])
def ondispatch_new():
    try:
        model = request.json.get('model')
        print(request.get_json())
        return dispatch(model, resolve_data_json(request.get_json()))
        
    except Exception as e:
        return jsonify({
            'code': 500,
            'message': str(e)
        })

In [13]:
# ===DEBUG===
print(modelHub.get_names())

dict_keys(['ConvNeXt_tiny'])


In [14]:
app = Flask("MLHub")
app.register_blueprint(api, url_prefix='/api')
app.run(port=_PORT_)

 * Serving Flask app "MLHub" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7001/ (Press CTRL+C to quit)
